In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "gpt2"  # Change this to "gpt2" if you get memory issues

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, add_prefix_space=False)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            return_dict_in_generate=True,
                                            pad_token_id=tokenizer.eos_token_id).to(device)

In [ ]:
prompt = tokenizer.decode(tokenizer.bos_token_id) + "."
input_ids = torch.tensor(tokenizer.encode(prompt)).reshape(1, -1).to(device)

with torch.no_grad():
    output = model(input_ids)
    logits = output[0]
    probs = torch.softmax(logits, dim=-1)

In [ ]:
import re
digits_probs = {}
for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    if re.fullmatch("[0-9]+", tokenizer.decode(i)):
        digits_probs[(tokenizer.decode(i)).replace(" ", "")] = prob

        
total_value = 0
one_digit = {str(x):0 for x in range(10)}
for key, value in digits_probs.items():
    one_digit[key[0]] += value
    total_value += value

for key, value in one_digit.items():
    one_digit[key] = value/total_value


    
print(one_digit)